## Codes de base

Ce document a pour but de présenter de manière condensée les différents codes utilisés pour implémenter le modèle d'ablation, uniforme ou non, à une ou deux dimensions. Par souci de concision, on ne montrera pas d'exemples ici mais juste les codes bruts, commentés autant que possible. Pour avoir des illustrations de ce que ça donne, on pourra se référer aux notebooks originaux plus fournis et plus bordéliques (il faut le dire). 

### Ablation uniforme unidimensionelle

#### Première version

Implémentation la plus simple possible du modèle. L'interface est représenté par un ensemble de points $(x,z)$. L'algorithme prend en entrée la liste des abscisses ($\textit{xx}$) et la liste des ordonnées ($\textit{zz}$) ; ainsi que la distance $\textit{l}$ d'érosion de l'interface. Il retourne la liste des abscisses et la liste des ordonnées correspondant à l'interface obtenu après érosion.

Dans un premier temps, l'algorithme calcule le vecteur normal, pointant vers le bas (intérieur du solide), en chaque point de l'interface. Ensuite, il propage chaque point de l'interface sur une distance $\textit{l}$ le long de ce vecteur normal pour obtenir la nouvelle interface. Cependant, si on s'arrête là, on voit apparaître des "boucles" au niveau des points de l'interface où le rayon de courbure est inférieur à $\textit{l}$. Ces boucles doivent être supprimées pour obtenir l'interface "physique" : c'est ce que fait l'antépénultième ligne du code. L'idée, c'est que supprimer ces boucles revient à supprimer tous les points situés à une distance strictement inférieure à $\textit{l}$ de l'interface initiale (c'est-à-dire, situés à une distance strictement inférieure à $\textit{l}$ d'au moins un des points de l'interface initiale).

Concrètement, $\textit{distance_matrix(new_coord, ex_coord)}$ retourne une matrice $D_{ij}$ contenant les distances entre chaque point de la nouvelle interface (indice $i$) et chaque point de l'ancienne interface (indice $j$). $\textit{distance_matrix(new_coord,ex_coord)<l*se}$ est une matrice dont l'indice $D^{'}_{ij}$ prend la valeur $\textit{True}$ si la distance entre le point $i$ de la nouvelle interface et le point $j$ de l'ancienne sont plus proche que $\textit{l}\cdot \textit{se}$, $\textit{False}$ sinon. On compare les distances à $\textit{l}\cdot \textit{se}$ plutôt qu'à $\textit{l}$ pour une histoire d'arrondis: si on ne le fait pas, le code considère parfois qu'un point de la nouvelle interface est strictement plus proche que $\textit{l}$ du point de l'ancienne interface dont il est issu (alors qu'il est théoriquement à une distance égale à $\textit{l}$, puisque c'est comme cela qu'on l'a construit) ; ce qui conduit à la suppression dudit point. Prendre $\textit{se}$ très proche de 1 mais strictement inférieur à 1 permet d'éviter ce désagrément. Ensuite, $\textit{np.unique(np.asarray((distance_matrix(new_coord,ex_coord)<l*se).nonzero()[0])}$ est une liste contenant l'indice $i$ s'il existe au moins un $j$ tel que $D^{'}_{ij}$ ait la valeur $\textit{True}$. Finalement, $\textit{np.delete(new_coord,np.unique(np.asarray((distance_matrix(new_coord,ex_coord)<l*se).nonzero()[0])),axis=0)}$ supprime tous les points dont les indices sont dans la liste précédente. On a, de la sorte, supprimé les boucles et l'algorithme finit par retourner la liste des abscisses et la liste des ordonnées de la nouvelle interface érodée.     

In [1]:
se=0.999999
def erod(xx,zz,l):
    #points obtained by moving by a distance l along the normal
    gradvec=np.array([np.gradient(zz),-np.gradient(xx)])#computation of a normal vector
    vecn=l*gradvec/np.linalg.norm(gradvec,axis=0)#normal vector of norm l
    xxprop=xx+vecn[0,:]
    zzprop=zz+vecn[1,:]#new points obtained     
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.transpose(np.array([xxprop,zzprop]))
    ex_coord=np.transpose(np.array([xx,zz]))
    cleaned_new_coord=np.delete(new_coord,np.unique(np.asarray((distance_matrix(new_coord,ex_coord)<l*se).nonzero()[0])),axis=0)
    final_xx,final_zz=cleaned_new_coord[:,0],cleaned_new_coord[:,1]
    return final_xx, final_zz

#### Version améliorée

La version de l'algorithme présentée ci-dessus présente l'inconvénient d'être assez coûteuse en temps de calcul, en particulier en raison de la matrice $\textit{distance_matrix}$ qui devient vite lourde à calculer quand le nombre de points de l'interface augmente. Or cette matrice calcule plein de distances inutiles : il est clair qu'il n'est pas nécessaire de calculer la distance entre le point le plus à gauche de l'interface initiale et le point le plus à droite de l'interface érodée pour savoir qu'elle est supérieure à $\textit{l}$. La version présentée ci-dessous améliore ainsi l'algorithme en ne calculant, pour un point de la nouvelle interface, que sa distance avec les $N_v$ voisins à droite et à gauche du point de l'interface initiale dont il est issu. On a donc remplacé le calcul de $\textit{distance_matrix}$, de taille $n \cdot n$ avec $n$ le nombre de points, par le calcul de la matrice $\textit{distance_voisins}$ de taille $n \cdot N_v$.

$N_v$ est un paramètre de la fonction qu'il s'agit de choisir avec soin. En effet, si $N_v$ est trop petit, on court le risque de garder des points qui devraient être enlevés (des points appartenant aux boucles dont on a parlé plus tôt) ; mais si $N_v$ est trop gros alors le code est trop lent. Une bonne manière de faire (à mon avis) consiste par exemple à choisir $N_v$ de la façon suivante : $N_v=2l/dx$ où $dx$ est l'écart selon l'axe x entre deux points successifs de l'interface initiale (c'est possible dans le cas où $dx$ est constant). En déterminant $N_v$ de la sorte, on est sûr de supprimer tous les points qui doivent l'être (cf inégalité triangulaire).

In [2]:
def distance_voisins(ex,new,n):
    distmat=np.zeros((2*n+1,np.shape(new)[1]))
    for k in range(-n,n+1):
        ex_rollk=np.roll(ex,k)
        distmat[n+k]=np.sqrt(np.sum((new-ex_rollk)**2,axis=0))
    return np.concatenate((distmat[:n],distmat[n+1:]),axis=0)   

def erod_improved(xx,zz,l,n):
    #points obtain by moving by a distance l along the normal
    gradvec=np.array([np.gradient(zz),-np.gradient(xx)])# Calcul du vecteur normal initial.
    vecn=l*gradvec/np.linalg.norm(gradvec,axis=0)
    xxprop=xx+vecn[0,:]
    zzprop=zz+vecn[1,:]    
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.array([xxprop,zzprop])
    ex_coord=np.array([xx,zz])
    cleaned_new_coord=np.delete(new_coord,np.unique((distance_voisins(ex_coord,new_coord,n)<l*se).nonzero()[1]),axis=1)
    final_xx,final_zz=cleaned_new_coord[0,:],cleaned_new_coord[1,:]
    return final_xx, final_zz

### Ablation non-uniforme unidimensionnelle

L'algorithme ci-dessous est une généralisation de l'algorithme précédent et permet que l'abalation ne soit plus uniforme. La distance d'érosion, qui était auparavant la même partout (on l'avait notée $\textit{l}$), est désormais a priori différente pour chaque point ($\textit{l}=\textit{l(i)}$). Concrètement, au lieu de multiplier la liste des vecteurs normaux par un scalaire $\textit{l}$, on la multiplie terme à terme avec la liste $\textit{l} \cdot f_{er}(xx,zz,e)$, où $f_{er}(xx,zz,e)$ est une fonction qui à chaque point de l'interface attribue une distance d'érosion dépendant a priori de l'interface entier et d'un paramètre scalaire $e$. Cette fonction est à renseigner en entrée du code par l'utilisateur, on peut imaginer plein de choses plus ou moins tordues.

La suppression des boucles, dans la deuxième partie du code, est légèrement plus subtile que dans le cas uniforme. Pour garder un point, il ne suffit plus de vérifier qu'il est strictement plus loin que $\textit{l}$ de l'interface initiale ; mais il faut qu'il soit plus loin que $l(i)$ de chaque point $i$ de l'interface initiale, ce qui demande de modifier le code.

In [ ]:
def erod_non_uniform_improved(xx,zz,l,f_er,e,n):
    ff=f_er(xx,zz,e)
    #points obtain by moving by a distance l along the normal
    gradvec=np.array([np.gradient(zz),-np.gradient(xx)])
    vecn=l*ff*gradvec/np.linalg.norm(gradvec,axis=0)
    xxprop=xx+vecn[0,:]
    zzprop=zz+vecn[1,:]    
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.array([xxprop,zzprop])
    ex_coord=np.array([xx,zz])
    f_er_mat=np.array([np.concatenate((np.roll(ff,n-i)[:n],np.roll(ff,n-i)[n+1:2*n+1])) for i in range(len(xxprop))]).transpose()
    cleaned_new_coord=np.delete(new_coord,np.unique((distance_voisins(ex_coord,new_coord,n)<l*se*f_er_mat).nonzero()[1]),axis=1)
    final_xx,final_zz=cleaned_new_coord[0,:],cleaned_new_coord[1,:]
    return final_xx, final_zz

### Ablation uniforme bidimensionnelle

L'algorithme ci-dessous est une extension de l'algorithme d'ablation uniforme unidimensionnelle à 2 dimensions : l'interface est désormais une surface $z(x,y)$. Le principe de l'algorithme est globalement similaire au principe de la fonction $\textit{erod_improved}$. Pour supprimer les boucles, on regarde la distance entre chaque point de la nouvelle interface et les voisins ($nx$ dans la direction x, $ny$ dans la direction y) du point de l'interface initiale dont ce point est issu. Toutes ces distances sont contenues dans le tableau à trois dimensions renvoyé par le fonction $\textit{distance_voisins_2d}$. 

Concrètement, la surface initiale est un ensemble de points situés sur une grille, laquelle peut être reconstituée à partir des deux listes $xx$ et $yy$ correspondant aux coordonnées des points de la grille dans les directions $x$ et $y$, respectivement. L'algorithme prend en entrée ces deux listes ainsi que la matrice $zz$ correspondant aux hauteurs des points sur ladite grille. Il prend aussi en entrée la distance d'érosion $l$ et le nombre de voisins $nx$ et $ny$ sur lesquels faire la comparaison. 

À la fin, le code renvoie la matrice $zer$ correspondant aux hauteurs de l'interface érodée sur la même grille que la grille d'entrée ; pour cela, il effectue une interpolation de Delaunay. 

In [ ]:
def distance_voisins_2d(ex,new,dx,dy,nx,ny):
    distmat=np.zeros((2*nx+1,2*ny+1,len(ex)))
    for k in range(-nx,nx+1):
        for j in range(-ny,ny+1):
            ex_roll_kj=np.roll(ex,-dx*j-k,axis=0)
            distmat[nx+k,ny+j]=np.sqrt(np.sum((new-ex_roll_kj)**2,axis=1))
    return distmat

def propag_voisins_2d(ff,lx,ly,nx,ny):
    voismat=np.zeros((2*nx+1,2*ny+1,len(ff)))
    for k in range(-nx,nx+1):
        for j in range(-ny,ny+1):
            ff_roll_kj=np.roll(ff,-lx*j-k)
            voismat[nx+k,ny+j]=ff_roll_kj
    return voismat   

def erod_improved_3D(xx,yy,zz,nx,ny,l):
    #initialization
    dx,dy=len(xx),len(yy)
    XX,YY=np.meshgrid(xx,yy)
    #points obtain by moving by a distance l along the normal
    gradvec=np.append(np.gradient(zz.transpose(),xx,yy),[0*zzt-1],axis=0)#normal vector
    vecn=l*gradvec/np.linalg.norm(gradvec,axis=0)
    xxprop=XX+vecn[0,:].transpose()
    yyprop=YY+vecn[1,:].transpose()
    zzprop=zz+vecn[2,:].transpose()
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.transpose(np.array([xxprop.flatten(),yyprop.flatten(),zzprop.flatten()]))
    ex_coord=np.transpose(np.array([XX.flatten(),YY.flatten(),zz.flatten()]))
    cleaned_new_coord=np.delete(new_coord,np.unique((distance_voisins_2d(ex_coord,new_coord,dx,dy,nx,ny)<l*eps).nonzero()[2]),axis=0)
    #return points on a regular grid by interpolating
    interpolator = tri.LinearTriInterpolator(tri.Triangulation(cleaned_new_coord[:,0],cleaned_new_coord[:,1]), cleaned_new_coord[:,2])
    return interpolator(XX, YY)

def erod_nonuniform_improved_3D2(xx,yy,zz,nx,ny,l,fer,pp):
    #initialization
    lx,ly=len(xx),len(yy)
    XX,YY=np.meshgrid(xx,yy)
    #points obtain by moving by a distance l along the normal
    gradvec=np.append(np.gradient(zz.transpose(),xx,yy),[0*zz.transpose()-1],axis=0)#normal vector 
    vecn=l*fer(XX,YY,zz,pp).transpose()*gradvec/np.linalg.norm(gradvec,axis=0)
    xxprop=XX+vecn[0,:].transpose()
    yyprop=YY+vecn[1,:].transpose()
    zzprop=zz+vecn[2,:].transpose()
    #keep only points which are at a distance farther than l from the initial curve
    new_coord=np.transpose(np.array([xxprop.flatten(),yyprop.flatten(),zzprop.flatten()]))
    ex_coord=np.transpose(np.array([XX.flatten(),YY.flatten(),zz.flatten()]))
    ff=fer(XX,YY,zz,pp).flatten()
    cleaned_new_coord=np.delete(new_coord,np.unique((distance_voisins_2d(ex_coord,new_coord,lx,ly,nx,ny)<l*eps*propag_voisins_2d(ff,lx,ly,nx,ny)).nonzero()[2]),axis=0)
    #return points on a regular grid by interpolating
    interpolator = tri.LinearTriInterpolator(tri.Triangulation(cleaned_new_coord[:,0],cleaned_new_coord[:,1]), cleaned_new_coord[:,2])
    return interpolator(XX, YY)